<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

# Dynamique

In [ ]:
%load_ext autoreload
%autoreload 2
from plot import *

## Intégration en temps (Newmark) sur Barre 1D

Prenons par exemple une série de barres connectées, discrétisée par 500 nœuds, avec $E = \rho = h = 1$ 

- Sa matrice de raideur est donc de la forme:

$$K = \begin{bmatrix}
1 & -1 \\
-1 & 2 & -1 \\
& \ddots &  \ddots & \ddots\\
& & -1 & 2 & -1 \\
& & & -1 & 1  \\
\end{bmatrix}$$

- Sa matrice de masse est donc:

$$M = \begin{bmatrix}
1/3 & 1/6 \\
1/6 & 2/3 & 1/6 \\
& \ddots &  \ddots & \ddots\\
& & 1/6 & 2/3 & 1/6 \\
& & & 1/6 & 1/3  \\
\end{bmatrix}$$


*Remarque exercice: essayer de retrouver d'ou viennent ces formes matricielles*

In [ ]:
def calculeK(nb_nodes):
    # Matrice de rigidité
    K = np.fromfunction(lambda i, j: (i == j)*2. + (i == j+1)
                            * (-1.) + (i+1 == j)*(-1.), (nb_nodes, nb_nodes))
    K[0, 0] = 1.
    K[-1, -1] = 1.
    return K

def calculeM(nb_nodes):    
    # Matrice de masse 
    M = np.fromfunction(lambda i, j: (i == j)*2./3. + (i == j+1)
                        * 1./6. + (i+1 == j)*1./6., (nb_nodes, nb_nodes))
    M[0, 0] = 1./3.
    M[-1, -1] = 1./3.
    return M

K = calculeK(5)
M = calculeM(5)

plot_matrix(K, 'K')
plot_matrix(M, 'M')

On donne une condition initiale en déplacement de la forme:

$$u(x,0) = \sin\left(\frac{x}{50}\right) \cdot \exp\left[-\left(\frac{x}{50}\right)^2\right]$$

In [ ]:
# nombre de noeuds
nb_nodes = 500
# node positions
nodes = np.arange(-nb_nodes/2, nb_nodes/2, dtype=float) 
# condition initiale
disp_init = lambda X: np.sin(X/50)*np.exp(-(X/50)**2) 
# calcule de K et M
K = calculeK(nb_nodes)
M = calculeM(nb_nodes)

# plot the initial displacement field
plt.plot(nodes, disp_init(nodes))

Les positions, déplacements, vitesses, et forces sont directement calculés à l'aide de la fonction `makeEvolution`.

In [ ]:
nsteps = 500   # nombre d'itérations
dt = 1         # durée d'un pas de temps


# calcul de l'évolution en temps
disp,vel,force = makeEvolution(nodes, nsteps, dt=dt, U=disp_init, K=K, M=M)

In [ ]:
# On peut observer l'évolution des déplacements.

spring_animation(nodes, disp, ylim=(-0.5, 0.5)) # 'ylim' permet de changer le niveau de zoom

#### Question: 

Regardez l'évolution des vitesses et des forces. Cela vous semble-t-il cohérent ?

In [ ]:
###########
# Solution:
##########


video1 = spring_animation(nodes, vel, ylim=(-0.02, 0.02))
video2 = spring_animation(nodes, force, ylim=(-0.001, 0.001))

display(video1)
display(video2)

# vitesse et force sont des déphasages (par dérivation) du déplacement. 
# On peut donc voir que la vitesse est anti-symmétrique au déplacement
# et les accelerations sont en phase avec les déplacements

### Mesure de l'énergie

La mesure de l'énergie est un des moyens indispensables pour vérifier la bonne **stabilité** du code. Lorsqu'il n'y a aucun travail extérieur appliqué l'énergie totale doit être conservée:

$$E^{tot} = E^{cin} + E^{pot}$$

Avec:
- l'énergie potentielle: $$E^{pot} = \frac{1}{2} \{d\} [K] \{d\}$$
- l'énergie cinétique : $$E^{cin} = \frac{1}{2} \{\dot{d}\} [M] \{\dot{d}\}$$

#### Question

Programmez une fonction qui retourne les énergies potentielles et cinétiques en fonction d'un champ de vitesse, d'un champ de déplacement et des matrices de masse $M$ et de raideur $K$ (qui sont données plus bas).

In [ ]:
# Matrice de rigidité (k = 1)


def computeEnergy(displacements, velocities):   
    # you have to write the code accordingly    
    epot = None
    ekin = None
    return epot, ekin

In [ ]:
###########
# Solution:
##########


def computeEnergy(U, V):
    epot = 0.5*U.T@K@U
    ekin = 0.5*V.T@M@V
    return epot, ekin

#### Question

Le code qui suit permet de tracer les différentes énergies en fonction du temps, par l'intermédiaire de la fonction _computeEnergy_. Jugez-vous la méthode choisie comme stable ?

In [ ]:
plotEnergyEvolution(disp, vel, computeEnergy)

### Stabilité de la méthode de Newmark-$\beta$

Le schémas implicite utilisé ($\beta = 1/4$, $\gamma = 1/2$) est inconditionnellement stable. 

On va chercher a implémenter le schemas explicite, i.e. $\beta = 0$, $\gamma = 1/2$ qui est conditionnellement stable:

$$
\begin{align}
1.& \quad \dot{d}_i^{\star} = \dot{d}_i + \frac{\Delta t}{2} \ddot{d}_i \\
2.& \quad d_{i+1} = d_i + \Delta t \dot{d}^{\star}_i \\
3.& \quad \ddot{d}_{i+1} = [M]^{-1}\left(-[K] d_{i+1} + F_{i+1}\right) \\
4.& \quad \dot{d}_{i+1} = \dot{d}_i^{\star} + \frac{\Delta t}{2} \ddot{d}_{i+1} \\
\end{align}
$$

#### Question

Complétez l'algorithme dans la cellule ci-dessous

In [ ]:
def NewMarkIntegrationExplicite(U, V, A, M, K, dt):
    # implementez le step 1, 
    # implémentez le step 2
    A[:] = scipy.sparse.linalg.spsolve(M, -K@U) # voila le step 3
    # implementez le step 4

In [ ]:
###########
# Solution:
##########


def NewMarkIntegrationExplicite(U, V, A, M, K, dt):
    V_star = V + dt/2*A
    U[:] = U + dt*V_star
    A[:] = scipy.sparse.linalg.spsolve(M, -K@U) 
    V[:] = V_star + dt/2*A

Ce schémas est stable lorsqu'il satisfait la condition de Courant–Friedrichs–Lewy (CFL):
  
$$\Delta t < \alpha \frac{h}{c}$$ 

avec $c = \sqrt{\frac{E}{\rho}}$, $h$ la taille de maillage et $\alpha$ un facteur de sécurité

#### Question

Trouvez le coefficient de sécurité $\alpha$ à l'aide de la cellule ci dessous

In [ ]:
nsteps = 100   # nombre d'itérations
dt = 1         # durée d'un pas de temps
alpha = .57      # trouvez la valeur de alpha

# calcule de l'évolution en temps
disp,vel,force = makeEvolution(nodes, nsteps, dt=dt*alpha, U=disp_init, time_integration=NewMarkIntegrationExplicite, K=calculeK, M=calculeM)

In [ ]:
plotEnergyEvolution(disp, vel, computeEnergy)

---

 **Solution:**

 ---



Avec $E = \rho = h = 1$ on a $c = 1$ et le pas de temps critique est $\Delta t = 1$. En manipulant le pas de temps on peut trouver que $\alpha \simeq 0.57$

#### Question 

Lisez le code de la fonction makeEvolution dans le fichier plot.py.
Ecrivez dans la cellule ci dessous une boucle d'intégration en temps avec des forces externes appliquées

In [ ]:
###########
# Solution:
##########


V = np.zeros_like(nodes)  # velocities
A = np.zeros_like(nodes)  # accelerations
U = disp_init(nodes)      # displacements
F = disp_init(nodes)      # external forces

# make the matrices sparse
K = scipy.sparse.csr_matrix(K)
M = scipy.sparse.csr_matrix(M)

nstep = 1000
for s in range(0, nsteps):
    V_star = V + dt/2*A
    U[:] = U + dt*V_star
    # it is possible to enforce boundary conditions by changing U
    # U[blocked] = 0
    
    # It is possible to alter F as a fonction of time
    # F[:] = ...
    A[:] = scipy.sparse.linalg.spsolve(M, -K@U+F) 
    V[:] = V_star + dt/2*A

## Fréquences naturelles et modes propres

### Exemple barre 1D

- Dans le cas d'une barre 1D
- Discretisation avec 100 éléments

In [ ]:
# nombre de noeuds
nb_nodes = 100 
# node positions
nodes = np.arange(-nb_nodes/2, nb_nodes/2, dtype=float) 
# calcule de K et M
K = calculeK(nb_nodes)
M = calculeM(nb_nodes)

Pour étudier les modes propres, il faut résoudre une équation du type:

$$[M]^{-1}[K] \{v\} = \lambda \{v\} \qquad \Rightarrow \qquad [K] \{v\} = \omega^2 [M]\{v\}$$

avec la fréquence telle que $\omega = 2\pi f$

En python la routine [scipy.sparse.linalg.eigs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html) peut résoudre ce type de problèmes sur des matrices creuses (plus rapide donc):

In [ ]:
def makeModalAnalysis(n_modes, K, M):

    #transformation en matrice creuse de M et K
    M = scipy.sparse.csr_matrix(M)
    K = scipy.sparse.csr_matrix(K)

    eigs = scipy.sparse.linalg.eigs(K, k=n_modes, M=M, which='SM')
    return eigs

#### Question

Utilisez la routine ci dessus pour calculer les trois premiers modes et pour les visualiser avec `plt.plot`

In [ ]:
###########
# Solution:
##########


eigen_values, eigen_vectors = makeModalAnalysis(3, K, M)

for eig_val, eig_vec in zip(eigen_values, eigen_vectors.T):
    # plot the initial displacement field
    if eig_val.real < 0:
        omega = 0
    else:
        omega = np.sqrt(eig_val.real)
    plt.plot(nodes, eig_vec.real, label=f'$\omega = {omega:.3f}$')
plt.legend(loc='best')

#### Question

Les modes propres sont sensibles aux conditions aux bords. Modifiez la routine `makeModalAnalysis`
pour bloquer les deux noeuds extrêmes ($x = 0$ et $x = L$) et recalculer les modes propres.

In [ ]:
###########
# Solution:
##########
 

def makeModalAnalysis(n_modes, K, M):

    # extrait les matrices effectives sans les noeuds bloques
    # on peut le faire condition homogene (encastrement)
    K = K[1:,  1:][:-1, :-1]
    M = M[1:,  1:][:-1, :-1]

    #transformation en matrice creuse de M et K
    M = scipy.sparse.csr_matrix(M)
    K = scipy.sparse.csr_matrix(K)

    eigs = scipy.sparse.linalg.eigs(K, k=n_modes, M=M, which='SM')
    return eigs

# calcul des modes
eigen_values, eigen_vectors = makeModalAnalysis(3, K, M)

# affichage
for eig_val, eig_vec in zip(eigen_values, eigen_vectors.T):
    # plot the initial displacement field
    if eig_val.real < 0:
        omega = 0
    else:
        omega = np.sqrt(eig_val.real)
    plt.plot(nodes[1:-1], eig_vec.real, label=f'$\omega = {omega:.3f}$')
plt.legend(loc='best')

Chaque mode est en faite un déplacement qui évolue dans le temps sous la forme analytique:

$$d(t) = v_\omega \cos(\omega t)$$

Avec $v_\omega$ le vecteur propre associé à la pulsation $\omega$.

La routine ci-dessous permet de générer une vidéo montrant les différents modes

In [ ]:
eigs = makeModalAnalysis(5, K, M)

anim = modesAnimation(nodes, ylim=(-0.2, 0.2), eigs=eigs)
ph.display_animation(anim)

#### Question

Changez la discrétisation pour n'avoir que $10$ noeuds. Que constatez vous ?

In [ ]:
###########
# Solution:
##########


nb_nodes = 100
nodes = np.arange(-nb_nodes/2, nb_nodes/2, dtype=float) 
K = calculeK(nb_nodes)
M = calculeM(nb_nodes)

eigs = makeModalAnalysis(5, K, M)

# calcul des modes
eigen_values, eigen_vectors = makeModalAnalysis(3, K, M)

# affichage
for eig_val, eig_vec in zip(eigen_values, eigen_vectors.T):
    # plot the initial displacement field
    if eig_val.real < 0:
        omega = 0
    else:
        omega = np.sqrt(eig_val.real)
    plt.plot(nodes[1:-1], eig_vec.real, 'o-', label=f'$\omega = {omega:.3f}$')
plt.legend(loc='best')

########
# La discrétisation influence les modes
#######